# Features Extraction & Clustering. Part II.
This part is focused on clustering inside top-levels clusters that were obtained in the previous part.

## Preparing

Firstly we have to re-run code of Part I before continue.

In [1]:
# Import of necessary libs and our classes
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import DBSCAN
from cj_loader import Storer, Extractor, extract_features
import warnings
warnings.filterwarnings("ignore")

# Init storer object with given data and calculate precense of data
storer = Storer()
precense = storer.applicability()

# Calculation of data completeness matrix and weights
data_compl = storer.data_completeness()
weights = data_compl.mean().sort_values()

# transform boolean matrix to numeric
weighted = precense.copy()
for index, row in precense.iterrows():
    weighted.loc[index][:] = row * weights

# calculate distances between pairs of coins
distances = pd.DataFrame(squareform(pdist(weighted)), index=weighted.index, columns=weighted.index)

# Top-level clustering with DBSCAN algorithm
clustering = DBSCAN(eps=0.3, min_samples=3).fit(distances)
labels = clustering.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

# Clusters aggregation
weighted['label'] = labels
clusters = {}
for label in np.unique(labels):
    cl = weighted[weighted['label'] == label]
    clusters[label] = cl

# Extracting features
cl_coin_features = {}
unclustered_coins = set()
for label, cluster in clusters.items():
    if label != -1:
        cl_coin_features[label] = extract_features(storer, '2014-04-01', coins_set=cluster.index)
    else:
        unclustered_coins.update(cluster.index.values.tolist())

Estimated number of clusters: 4


## Adjusting clustering parameters

DBSCAN is one of the most common clustering algorithms and also most cited in scientific literature. It has a lot of advantages and widely recommended for various tasks. One of disatvantages of DBSCAN is routine of parameters selection, especially epsilon parameter. In the previous part DBSCAN was applied to data having all elements in \[0;1\] range. That simplified choosing of epsilon. Current clustering task is significantly more complicated. Parameters are non-normalized and have it's own scales. Number of parameters is pretty big so the task can be considered as high-dimensional clustering. High number of dimensions will also make direct clusters visualization impossible. 

Best possible approach to choose correct value for epsilon is to research results obtained with different values. Fortunately, such method was developed and described in several publications.

[HDBSCAN](https://hdbscan.readthedocs.io/en/latest/index.html) - Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
import hdbscan

for top_level_cl in range(n_clusters_):
    print("Top-level cluster: {}".format(top_level_cl))
    tc = cl_coin_features[top_level_cl]
    tc = tc.replace([np.inf, -np.inf], np.nan)
    tc = tc.dropna(axis='columns', how='any')
    print("shape: {}".format(tc.shape))
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2)
    ll_labels = clusterer.fit_predict(tc)
    tc['label'] = ll_labels
    for label in np.unique(ll_labels):
        if label != -1:
            print('label %d' %label)
            cl = tc[tc['label'] == label]
            display(cl)
            print('-'*40)
        else:
            noise_coins = tc[tc['label'] == label].index.values.tolist()
            unclustered_coins.update(noise_coins)
            
    print('='*40) 
    
print("Unclustered coins:")
for co in unclustered_coins:
    print(co)

Top-level cluster: 0
shape: (27, 95)
label 0


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
vechain,949.394509,699.5,1200.307321,9.380132,127.998701,2.590254,2.61,2.073741,0.348733,-0.837170,...,13806.638918,57591.330938,11.817810,172.859691,1.080835,1.081693,0.114190,-0.055933,0.398785,0
icon,1882.450161,1056.0,1919.075742,1.637988,1.567113,3.354226,2.67,2.305503,1.513255,1.962505,...,15439.828774,555552.458802,9.788784,101.536048,1.067163,1.020315,0.210151,0.692688,-0.153612,0


----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
binance-coin,999.766234,217.0,7791.168444,14.382106,226.810603,8.084689,9.26,5.893013,0.043746,-1.360099,...,5051.448981,23890.993978,6.502648,49.615987,1.082846,1.062092,0.193961,0.827556,2.178119,1
omisego,3904.139896,2467.5,8247.695864,6.691323,49.809485,10.894890,9.82,4.869719,0.360876,0.131471,...,7600.454755,18506.548801,13.566305,214.684558,1.375349,1.256275,0.413903,4.556722,30.751261,1


----------------------------------------
label 2


...

----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
lisk,1256.345088,845.5,1306.419742,2.170264,8.431071,5.084455,1.765000,7.069427,1.846910,2.940810,...,1.780763,0.396637,1.179027,2.528794,234.195165,194.815368,189.347875,1.589927,4.444137,1
gas,4605.768519,4411.5,5576.554873,2.587282,15.329782,23.146736,20.590000,13.790248,1.121104,1.914183,...,1.800429,0.809647,1.883398,3.804189,73.458813,0.013154,197.234739,3.986829,21.984833,1
digibyte,7653.289331,4285.0,7987.288277,2.910194,12.477327,0.007896,0.000279,0.016372,2.991550,11.571083,...,0.640099,0.136387,0.030436,2.460600,1334.301447,122.844788,2645.075160,4.811180,40.258149,1
verge,4775.912040,707.0,9105.175995,2.395383,4.998431,0.011389,0.000025,0.029746,3.755064,16.750922,...,0.701068,0.183204,-0.552401,-0.252065,950.513747,33.238570,7896.868751,31.924672,1099.745328,1
zcash,53671.197802,56198.0,28965.458710,0.018248,-1.350792,233.724953,220.800000,184.993409,3.025848,20.671637,...,0.096124,0.051123,5.473586,41.712232,2.476253,1.655919,3.959993,4.433368,23.606822,1
waves,4530.174551,3220.0,4783.672582,2.243263,7.484182,3.155054,2.860000,3.291030,1.338110,1.970991,...,2.522097,3.895967,2.982409,12.366264,42.177654,4.054797,159.940076,5.716705,32.118045,1
dogecoin,44725.500591,35972.0,30990.339987,2.850689,11.276300,0.001132,0.000243,0.001916,3.192562,13.440000,...,0.445320,0.129465,0.957839,0.907888,37756.476612,24201.275609,332187.587638,40.333971,1632.795528,1
pivx,2817.088009,2696.0,1551.385042,1.526174,8.624566,2.080260,1.360000,2.711160,1.755510,3.364241,...,0.187977,0.169446,1.370168,3.121039,41.565378,1.146900,1076.473628,29.339617,858.875856,1
decred,10833.712222,11097.5,3816.079987,0.222284,0.659588,29.186445,15.510000,33.340191,0.980179,-0.305222,...,0.302313,0.072040,1.659293,9.949313,59.520629,8.812540,148.366603,5.126046,33.825512,1


----------------------------------------
label 2


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
nem,870.648026,356.0,1267.603028,4.200264,32.138797,0.131956,0.005698,0.247699,3.310615,13.703799,...,1.634948,20.710297,7.254711,70.183217,6197.273077,5089.033717,7475.500598,20.369103,578.018261,2
dash,18778.626667,10335.0,17916.087571,2.149134,12.087689,120.214314,7.775000,230.905655,2.654101,7.803201,...,0.170657,0.158673,3.688645,26.213098,8.365254,3.932487,10.370287,7.242054,139.679172,2


----------------------------------------
Unclustered coins:
qtum
cryptonex
mixin
kin
stellar
tether
nano
ripple
huobi-token
smartcash
gifto
mithril
nxt
elastos
monero
factom
tron
ethereum
stratis
tezos
bitcoin
iota
veritaseum
ontology
litecoin
pundi-x
iostoken
bitcoin-cash
eos
nebulas-token
cardano
tenx


## Important notes
### Approach limitations
Implemented approach is very general. It doesn't detect any mutual dependencies, following trends and other complex things. So it should be considered as clustering by basic time series characteristics. Particularly, top-level clustering is rather necessary measure then usual taken step. List of features describing each time serie is quite short, but it can easily be extended with additional features.
### Label "-1"
HDBSCAN (as extension of DBSCAN method) groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). This leads to appearing a section named "Unclustered coins" which includes these outliers belonging to -1 label. This list consists of two types of coins: coins with big amount of data missed and **real** outliers. Each coin presented in this list should be considered as **special**. it's not surprising, therefore, that couple of crypto-headliners appeared there: they are really outliers by many of parameters.
### Metrics
HDBSCAN [can use different metrics][0] to calculate distance between coins by their values of features. The case examined uses default Euclidean metrics. Main disadvantage of such metrics is assumption that different parameters equally influence the distance. That actually is not true. The question of choosing metrics for high-dimensional clustering is [very][1] [controversial][2]. Generally one of the best ways is to define weight for each feature and use weighted Euclidean distance. Possibly this can be the theme of separate (and pretty complex) research.
### Methods
Of course there are a lot of clustering algorithms nowadays that can be applied for this task instead of (H)DBSCAN. But most probably, applying of described approach to data splitted by timeframes of different market states as well as widening feature-sets extraction method can deliver more significant results. Next proposed step is switching to tsfresh library for automated features extraction. 
[0]:https://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html#what-about-different-metrics
[1]:https://www.researchgate.net/post/What_is_the_best_distance_measure_for_high_dimensional_data
[2]:https://stats.stackexchange.com/questions/99171/why-is-euclidean-distance-not-a-good-metric-in-high-dimensions

# Clustering using automated features extraction technique
Module tsfresh is used to to extract characteristics from time series. Actual version for the moment extracts ~800 features out of the box. Since our data consists of 22 series for each coin (15 from csv files and 7 additionally constructed), total number of features describing each coin is 17468. This time the task dimensionality becomes even more higher.

## Preparing data
Tsfresh has [it's own format][1] of pandas.Dataframe() that should be used for features extraction. Since tsfresh [can easily handle][2] time series of unequal length, we can use **Storer.mf** containing the data across all coins, without using **Extractor** class for each coin separately. Additional method **tsfresh_format()** was implemented as @property of **Storer** class. This method converts mainframe to format compatible with tsfresh. 

[1]:https://tsfresh.readthedocs.io/en/latest/text/data_formats.html
[2]:https://tsfresh.readthedocs.io/en/latest/text/faq.html

In [4]:
# Import of tsfresh methods
from tsfresh import extract_features as tsf_ef
from tsfresh.utilities.dataframe_functions import impute

In [5]:
%%time

# Convert mainframe to format compatible with tsfresh
ts_df = storer.tsfresh_format

display(ts_df.head(10))
display(ts_df.tail(10))

,id,kind,time,value
1232,vechain,active_address,2017-08-15,2.00
1233,vechain,active_address,2017-08-16,159.25
1234,vechain,active_address,2017-08-17,316.50
1235,vechain,active_address,2017-08-18,473.75
1236,vechain,active_address,2017-08-19,631.00
1237,vechain,active_address,2017-08-20,205.00
1238,vechain,active_address,2017-08-21,393.00
1239,vechain,active_address,2017-08-22,296.00
1240,vechain,active_address,2017-08-23,470.00
1241,vechain,active_address,2017-08-24,456.00


,id,kind,time,value
3989174,aeternity,tx_per_address,2018-07-17,1.266904
3989175,aeternity,tx_per_address,2018-07-18,1.435262
3989176,aeternity,tx_per_address,2018-07-19,1.121864
3989177,aeternity,tx_per_address,2018-07-20,1.053191
3989178,aeternity,tx_per_address,2018-07-21,1.116022
3989179,aeternity,tx_per_address,2018-07-22,1.010638
3989180,aeternity,tx_per_address,2018-07-23,1.643068
3989181,aeternity,tx_per_address,2018-07-24,1.230769
3989182,aeternity,tx_per_address,2018-07-25,1.405858
3989183,aeternity,tx_per_address,2018-07-26,1.080169


Wall time: 3min 32s


## Features extraction
Run process of automated features extraction with tsfresh and save the result to csv file.

In [6]:
%%time

extracted_features = tsf_ef(ts_df, column_id="id", column_sort="time", column_kind="kind", column_value="value")
extracted_features = impute(extracted_features)
extracted_features.to_csv("ef.csv")  # Save it because it takes too long to calculate it each time

Feature Extraction: 100%|████████████████████████████████████████████████████████| 20/20 [26:14<00:00, 78.70s/it]


Wall time: 26min 30s


In [7]:
# Show the structure of dataframe with extracted features
display(extracted_features.head(10))
display(extracted_features.tail(10))

variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_-inf,volume__value_count__value_0,volume__value_count__value_1,volume__value_count__value_inf,volume__value_count__value_nan,volume__variance,volume__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
0x,6.055194e+08,82731.0,0.117297,0.106349,0.027818,2365.457143,-0.336061,-47.954622,23.395524,5323.250000,...,-1.052709e+21,-6.852882e+20,-2.590628e+19,0.0,0.0,0.0,0.0,0.0,3.565607e+14,1.0
aelf,3.080422e+08,73441.0,-0.004179,-0.004811,0.000163,3102.913043,-0.295909,-142.015810,100.039258,10451.800000,...,-8.534993e+21,-2.320098e+22,-4.200120e+22,0.0,0.0,0.0,0.0,0.0,2.931632e+15,1.0
aeternity,1.128107e+08,41060.0,0.037174,0.020251,0.009742,1155.857398,-0.189691,-22.828209,21.222047,3799.857143,...,2.524027e+20,-5.652238e+19,-7.643957e+20,0.0,0.0,0.0,0.0,0.0,1.330522e+14,1.0
aion,8.424410e+07,54263.0,0.065037,0.052556,0.012044,968.441379,0.002184,0.232020,20.441058,2087.619048,...,-2.047757e+20,-2.205417e+20,-1.225549e+20,0.0,0.0,0.0,0.0,0.0,6.144918e+13,1.0
ardor,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,-9.176908e+20,-2.808596e+21,-3.516748e+21,0.0,9.0,1.0,0.0,0.0,2.289649e+14,1.0
ark,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,-5.432228e+19,-7.823966e+18,6.011114e+19,0.0,0.0,0.0,0.0,0.0,4.445506e+13,1.0
augur,1.139342e+08,43618.0,0.350662,0.394984,0.021014,966.594595,-0.204435,-11.882883,9.617490,1601.666667,...,-2.457617e+21,-3.936387e+20,-1.350774e+21,0.0,1.0,2.0,0.0,0.0,1.510852e+14,1.0
bancor,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,1.982417e+20,1.819819e+20,-2.060632e+18,0.0,0.0,0.0,0.0,0.0,5.326489e+13,1.0
basic-attention-token,3.042960e+08,62326.0,0.530526,0.557664,0.029372,1202.082452,-0.136181,-6.645122,7.549721,1849.844444,...,-4.413956e+20,1.549729e+20,1.207532e+20,0.0,0.0,0.0,0.0,0.0,1.318699e+14,1.0


variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_-inf,volume__value_count__value_0,volume__value_count__value_1,volume__value_count__value_inf,volume__value_count__value_nan,volume__variance,volume__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
verge,1.466247e+11,924126.0,0.859942,0.860190,0.005304,-5607.959918,0.606455,172.635328,19.337607,-6691.889163,...,-2.074784e+23,-3.804209e+23,1.980499e+22,0.0,0.0,0.0,0.0,0.0,8.109743e+15,1.0
veritaseum,6.730167e+07,29287.0,0.476574,0.506777,0.031793,718.904718,-0.415149,-10.240210,3.383006,1151.327273,...,4.439179e+16,6.335611e+16,9.247988e+15,0.0,0.0,0.0,0.0,0.0,3.238983e+11,1.0
waltonchain,6.250837e+07,35426.0,0.215901,0.187632,0.009683,648.680162,-0.065799,-2.887406,7.297838,1380.833333,...,-9.191506e+20,-5.695716e+21,1.131337e+21,0.0,0.0,0.0,0.0,0.0,5.770324e+14,1.0
wanchain,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,-1.058998e+22,-3.770552e+21,-6.212197e+21,0.0,0.0,0.0,0.0,0.0,3.997042e+14,1.0
waves,2.660788e+10,1686466.0,0.437526,0.433732,0.008667,559.067669,0.615942,305.572888,50.455748,5259.109890,...,3.321996e+19,-1.878305e+20,-2.444620e+20,0.0,0.0,0.0,0.0,0.0,3.305474e+14,1.0
wax,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,-2.832079e+21,-2.581930e+21,-1.161918e+21,0.0,0.0,0.0,0.0,0.0,1.313925e+14,1.0
zcash,2.369382e+12,1906523.0,0.927693,0.934783,0.001058,13040.014904,0.854419,1482.187523,114.470386,21783.802198,...,-2.296494e+21,-3.196160e+22,-6.077346e+22,0.0,0.0,0.0,0.0,0.0,4.200767e+15,1.0
zcoin,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,5.917691e+19,-1.849818e+20,-1.092619e+20,0.0,0.0,0.0,0.0,0.0,3.737511e+13,1.0
zencash,2.608398e+09,259974.0,0.340423,0.353796,0.010806,1238.273728,-0.057492,-6.645122,36.135563,3664.750000,...,-9.596289e+19,-1.553048e+20,-1.018554e+20,0.0,0.0,0.0,0.0,0.0,1.789921e+13,1.0


## Normalization
It's clear that each feature has its own scale. So this features dataframe should be rescaled, otherwise features with larger scale will affects the distance measure higher. **normalize** function below will scale each feature to \[0, 1\] range. 

In [8]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [9]:
ef_norm = normalize(extracted_features)
display(ef_norm.head(10))

variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_-inf,volume__value_count__value_0,volume__value_count__value_1,volume__value_count__value_inf,volume__value_count__value_nan,volume__variance,volume__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
0x,1.282667e-06,0.001078,0.130591,0.155094,0.548957,0.448015,0.241708,0.292583,0.014595,0.321105,...,0.117465,0.712655,0.897814,NaN,0.000000,0.000000,NaN,NaN,0.000033,NaN
aelf,6.474441e-07,0.000942,0.000268,0.041724,0.000000,0.450795,0.266243,0.280244,0.064841,0.333218,...,0.117430,0.712579,0.897766,NaN,0.000000,0.000000,NaN,NaN,0.000273,NaN
aeternity,2.305537e-07,0.000469,0.044632,0.067284,0.190140,0.443457,0.331150,0.295880,0.013170,0.317506,...,0.117471,0.712657,0.897813,NaN,0.000000,0.000000,NaN,NaN,0.000012,NaN
aion,1.695535e-07,0.000662,0.074524,0.100232,0.235831,0.442750,0.448398,0.298905,0.012658,0.313462,...,0.117469,0.712656,0.897814,NaN,0.000000,0.000000,NaN,NaN,0.000006,NaN
ardor,5.559542e-06,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.117465,0.712648,0.897810,NaN,0.095745,0.012821,NaN,NaN,0.000021,NaN
ark,5.559542e-06,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.117469,0.712657,0.897814,NaN,0.000000,0.000000,NaN,NaN,0.000004,NaN
augur,2.329527e-07,0.000506,0.380949,0.449468,0.413903,0.442743,0.322140,0.297315,0.005563,0.312314,...,0.117458,0.712656,0.897813,NaN,0.010638,0.025641,NaN,NaN,0.000014,NaN
bancor,5.559542e-06,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.117471,0.712658,0.897814,NaN,0.000000,0.000000,NaN,NaN,0.000005,NaN
basic-attention-token,6.394445e-07,0.000780,0.573911,0.615383,0.579813,0.443631,0.363848,0.298003,0.004207,0.312901,...,0.117468,0.712658,0.897815,NaN,0.000000,0.000000,NaN,NaN,0.000012,NaN


Some features were the same value for all coins. This may leads to appearing of **Inf**s and **NaN**s. To avoid this we should drop such features. Dropping will not affect the result because such equal features are non informative.

In [10]:
ef_norm = ef_norm.dropna(axis='columns', how='any')
ef_norm.shape

(115, 15634)

## Clustering
As previously we will use hdbscan to perform clustering. Since this time number of features is quite higher we can expect that most of coins will be considered as outliers.

In [11]:
tf_clusterer = hdbscan.HDBSCAN(min_cluster_size=2)
tf_labels = tf_clusterer.fit_predict(ef_norm)
ef_norm['label'] = tf_labels
for label in np.unique(tf_labels):
    if label != -1:
        print('label %d' %label)
        cl_tf = ef_norm[ef_norm['label'] == label]
        display(cl_tf)
        print('-'*40)

label 0


variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__sum_values,volume__symmetry_looking__r_0.05,volume__symmetry_looking__r_0.1,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_0,volume__value_count__value_1,volume__variance,label
id,,,,,,,,,,,,,,,,,,,,,
bitshares,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.006706,1.0,1.0,0.117462,0.712534,0.897765,0.0,0.0,1.005685e-04,0
neoscoin,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000093,1.0,1.0,0.117470,0.712657,0.897814,0.0,0.0,4.762230e-08,0
nxt,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.004531,1.0,1.0,0.117499,0.712691,0.897811,0.0,0.0,1.321656e-04,0


----------------------------------------
label 1


variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__sum_values,volume__symmetry_looking__r_0.05,volume__symmetry_looking__r_0.1,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_0,volume__value_count__value_1,volume__variance,label
id,,,,,,,,,,,,,,,,,,,,,
bitsend,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000092,1.0,1.0,0.117470,0.712657,0.897814,0.191489,0.448718,1.530308e-07,1
crown,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000024,1.0,1.0,0.117470,0.712657,0.897814,1.000000,0.679487,1.875903e-09,1
emercoin,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000265,1.0,1.0,0.117469,0.712657,0.897814,0.127660,0.076923,4.712445e-07,1
exclusivecoin,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000074,1.0,1.0,0.117470,0.712657,0.897814,0.031915,0.141026,7.142969e-08,1
korecoin,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000122,1.0,1.0,0.117470,0.712657,0.897814,0.542553,1.000000,1.493741e-07,1
monetaryunit,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000106,1.0,1.0,0.117470,0.712657,0.897814,0.234043,0.230769,8.807120e-08,1


----------------------------------------
label 2


...

----------------------------------------
label 4


variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__sum_values,volume__symmetry_looking__r_0.05,volume__symmetry_looking__r_0.1,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_0,volume__value_count__value_1,volume__variance,label
id,,,,,,,,,,,,,,,,,,,,,
ardor,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.001493,1.0,1.0,0.117465,0.712648,0.897810,0.095745,0.012821,2.132189e-05,4
ark,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000871,1.0,1.0,0.117469,0.712657,0.897814,0.000000,0.000000,4.139766e-06,4
cryptonex,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000098,0.0,0.0,0.117470,0.712657,0.897814,0.000000,0.000000,6.604458e-08,4
cybermiles,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.002618,0.0,1.0,0.117473,0.712654,0.897813,0.000000,0.000000,7.631944e-05,4
decentraland,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.001773,0.0,1.0,0.117467,0.712653,0.897813,0.000000,0.000000,3.272810e-05,4
digixdao,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.001666,1.0,1.0,0.117440,0.712639,0.897814,0.000000,0.000000,2.762521e-05,4
elastos,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.001574,1.0,1.0,0.117469,0.712656,0.897814,0.000000,0.000000,2.666893e-05,4
enigma-project,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000957,1.0,1.0,0.117467,0.712653,0.897813,0.000000,0.000000,1.422224e-05,4
factom,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.001574,1.0,1.0,0.117469,0.712656,0.897814,0.000000,0.000000,2.666893e-05,4


----------------------------------------
label 5


variable,active_address__abs_energy,active_address__absolute_sum_of_changes,"active_address__agg_autocorrelation__f_agg_""mean""","active_address__agg_autocorrelation__f_agg_""median""","active_address__agg_autocorrelation__f_agg_""var""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","active_address__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,volume__sum_values,volume__symmetry_looking__r_0.05,volume__symmetry_looking__r_0.1,volume__time_reversal_asymmetry_statistic__lag_1,volume__time_reversal_asymmetry_statistic__lag_2,volume__time_reversal_asymmetry_statistic__lag_3,volume__value_count__value_0,volume__value_count__value_1,volume__variance,label
id,,,,,,,,,,,,,,,,,,,,,
bitcoin-private,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000035,0.0,1.0,0.117470,0.712657,0.897814,0.0,0.0,2.214374e-08,5
hshare,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.008126,0.0,1.0,0.117411,0.712611,0.897814,0.0,0.0,3.221602e-04,5
moac,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000060,0.0,1.0,0.117470,0.712657,0.897814,0.0,0.0,2.658076e-07,5
polymath-network,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000306,1.0,1.0,0.117470,0.712657,0.897814,0.0,0.0,2.309527e-06,5
wanchain,0.000006,0.003667,0.369964,0.407461,0.211258,0.443767,0.411932,0.298003,0.022947,0.317187,...,0.000765,1.0,1.0,0.117420,0.712644,0.897807,0.0,0.0,3.722166e-05,5


----------------------------------------


## Conclusions
As it was expected, most of coins were considered outliers. The main reason for such behavior is very high dimensionality of features space. Actually we have ~150 points, each having it's own value for every of ~15k coordinates. And our goal is to find points that close enough in this rarefied space to be considered **cluster**. On the other hand this means that founded clusters represent coins that very similar. 

Part II of final report was successfully completed. Two different approaches were demonstrated. Both approaches, manual features extracion and automated feature extraction with tsfresh library, gave different interesting results as it was initially expected.

Next step is performing clustering with taking into account different market states.